# Pauli algebra
The single-qubit Pauli matrices $I, X, Y, Z$ form a group under multiplication.  Ignoring complex phase, this group is isomorphic to the Galois group with four elements.  Tensor products of Pauli matrices similarly form a group.
`paulimer` provides support for a variety of Pauli group operations, as demonstrated below.

In [1]:
import paulimer
from paulimer import PauliGroup, SparsePauli as Pauli

## Pauli basics

Before working with groups, let's explore the `Pauli` type which represents individual Pauli operators. These support multiplication (with phases), and commutativity checks.

In [2]:
x = Pauli("X")
y = Pauli("Y")
z = Pauli("Z")

print(f"X * Y = {x * y}")
print(f"Y * X = {y * x}")
print(f"X commutes with X: {x.commutes_with(x)}")
print(f"X commutes with Z: {x.commutes_with(z)}")

X * Y = 𝑖Z
Y * X = -𝑖Z
X commutes with X: True
X commutes with Z: False


Pauli operators are not limited to single qubits.  They can span and support any number of qubits.  Here is a Pauli that spans four qubits.

In [3]:
Pauli("IXYZ")

IXYZ

## Pauli groups
A `PauliGroup` object is initialized by providing the `Pauli` generators.

The generators can be retrieved by accessing the `generators` property.  The `support` property is the set of qubits on which the group has non-trivial support.

In [4]:
group = PauliGroup([Pauli("IX"), Pauli("ZZ")])
print(f"Generators: {group.generators}, support: {group.support}")

Generators: [IX, ZZ], support: [0, 1]


### Enumeration

`PauliGroup` provides a `log2_size` property for the group order (as a power of 2), and an `elements` property to iterate over all elements of the group (not just the generators).

In [5]:
print(f"Group size: 2**{group.log2_size} = {2**group.log2_size}")
print(list(group.elements))

Group size: 2**3 = 8
[I, -I, IX, -IX, ZZ, -ZZ, -𝑖ZY, 𝑖ZY]


### Comparison

Comparison operators `{<, <=, ==}` are available for evaluating group containment.  Note: the comparison operations operate modulo phases.
Group quotients can be obtained by using the division operator `/`.

In [6]:
subgroup = PauliGroup([Pauli("IX")])
print(f"subgroup < group: {subgroup < group}")
print(f"subgroup <= group: {subgroup <= group}")
print(f"subgroup == group: {subgroup == group}")
print(f"group / subgroup: {(group / subgroup)}")

subgroup < group: True
subgroup <= group: True
subgroup == group: False
group / subgroup: ⟨I, ZZ⟩


### Group Operations

Groups support membership testing with `in`, intersection with `&`, union with `|`, and factorization of elements into generators.

In [7]:
print(f"IXI in group: {Pauli('IX') in group}")
print(f"ZXI in group: {Pauli('ZX') in group}")
print(f"YII in group: {Pauli('YI') in group}")

product = Pauli("iZY")
factorization = group.factorization_of(product)
print(f"\nFactorization of {product} in group: {factorization}")

group_a = PauliGroup([Pauli("XI"), Pauli("IZ")])
group_b = PauliGroup([Pauli("XI"), Pauli("ZI")])
print(f"\nGroup A: {group_a}")
print(f"Group B: {group_b}")
print(f"A ∩ B: {(group_a & group_b).generators}")
print(f"A ∪ B: {(group_a | group_b).generators}")

IXI in group: True
ZXI in group: False
YII in group: False

Factorization of 𝑖ZY in group: [IX, ZZ, -I]

Group A: ⟨X, IZ⟩
Group B: ⟨X, Z⟩
A ∩ B: [X, I]
A ∪ B: [X, IZ, X, Z]


### Properties and canonical forms

Other group properties and forms including rank, commutivity, etc. can be computed by standalone functions.

In [8]:
print(f"Binary rank: {group.binary_rank}")
print(f"Is abelian: {group.is_abelian}")
print(f"Standard generators: {group.standard_generators}")
print(f"Symplectic form: {list(paulimer.symplectic_form_of(group.generators))}")

Binary rank: 2
Is abelian: False
Standard generators: [IX, ZZ]
Symplectic form: [ZZ, IX]


### Stabilizer Formalism

In quantum error correction, stabilizer codes are defined by abelian Pauli groups. The $[\![4,2,2]\!]$ code encodes 2 logical qubits into 4 physical qubits, with stabilizers $\langle XXXX, ZZZZ \rangle$. The **logical operators** are the centralizer modulo the stabilizers.

In [9]:
code_422 = PauliGroup([Pauli("XXXX"), Pauli("ZZZZ")])
print(f"[[4,2,2]] stabilizers: {code_422}")
print(f"Is stabilizer group: {code_422.is_stabilizer_group}")

centralizer = paulimer.centralizer_of(code_422, supported_by=[0, 1, 2, 3])
print(f"Centralizer: {centralizer}")

print(f"Centralizer symplectic form: {list(paulimer.symplectic_form_of(centralizer.generators))}")

[[4,2,2]] stabilizers: ⟨XXXX, ZZZZ⟩
Is stabilizer group: True
Centralizer: ⟨XX, XIX, XIIX, ZZ, ZIZ, ZIIZ, 𝑖I⟩
Centralizer symplectic form: [ZIIZ, XX, ZZ, IXX, 𝑖I, XXXX, ZZZZ]


## Interactive Explorer

Build your own Pauli group by adding generators!

In [10]:
import widgets
widgets.pauli_group_explorer()